In [1]:
import os

import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None

# Data Cleaning
## Import the data
Start by finding a list of all data files (ensuring that the found path is relitive to where everything's run from).

Then combine all seperated data files into single CSVs!

**Note that in the future this code may be simplified by using glob instead of os.listdir**

In [2]:
energy_locations = os.listdir("../Data/Energy")
temperature_locations = os.listdir("../Data/Temperature")

energy_CSVs = [pd.read_csv("../Data/Energy/" + location) for location in energy_locations]
temperature_CSVs = [pd.read_csv("../Data/Temperature/" + location) for location in temperature_locations if "Data" in location]

C:\Users\Javis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,22,24,26,28,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Javis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,28,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Javis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Javis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (12,16,22,24,26,28,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, res

In [3]:
energy_data = pd.concat(energy_CSVs, ignore_index=True)
temperature_data = pd.concat(temperature_CSVs, ignore_index=True)

# Note some excess data is shaved off here for simplicity
stations_data = pd.read_csv("../Data/Temperature/HM01X_StnDet_999999999743964.txt", skiprows=5, header=None, usecols=range(1, 21), names=["StationNumber", "RainfulDistrict", "Station", "StationOpenDate", "StationCloseDate", "Latitude", "Longitude", "LocationMethod", "State", "StationHeight", "BarometerHeight", "WMO", "FirstYear", "LastYear", "CompletionPercent", "YQualityPercent", "NQualityPercent", "WQualityPercent", "SQualityPercent", "IQualityPercent"])

## Column standardising
Start by renaming all columns in a standardised way (writen in simplest form with no spaces and first letters capatilised)
*Note this standarisation hapens whilst importing the station details (so unecessary here)!*

In [4]:
energy_data.dtypes
temperature_data.dtypes
stations_data.dtypes

REGION             object
SETTLEMENTDATE     object
TOTALDEMAND       float64
RRP               float64
PERIODTYPE         object
dtype: object

hm                                                         object
Station Number                                              int64
Year Month Day Hour Minutes in YYYY                         int64
MM                                                          int64
DD                                                          int64
HH24                                                        int64
MI format in Local time                                     int64
Year Month Day Hour Minutes in YYYY.1                       int64
MM.1                                                        int64
DD.1                                                        int64
HH24.1                                                      int64
MI format in Local standard time                            int64
Precipitation since 9am local time in mm                   object
Quality of precipitation since 9am local time              object
Air Temperature in degrees C                               object
Quality of

StationNumber          int64
RainfulDistrict       object
Station               object
StationOpenDate       object
StationCloseDate      object
Latitude             float64
Longitude            float64
LocationMethod        object
State                 object
StationHeight        float64
BarometerHeight      float64
WMO                    int64
FirstYear              int64
LastYear               int64
CompletionPercent      int64
YQualityPercent        int64
NQualityPercent        int64
WQualityPercent        int64
SQualityPercent        int64
IQualityPercent        int64
dtype: object

In [5]:
energy_data.columns = ["Region", "Date", "TotalDemand", "RRP", "PeriodType"]
temperature_data.columns = [
    "HM", "StationNumber", "Year1", "Month1", "Day1", "Hour1", "Minute1", "Year", "Month", "Day", "Hour", "Minute", "Precipitation", "PrecipitationQuality",
    "AirTemperature", "AirTemperatureQuality", "WetBulbTemperature", "WetBulbTemperatureQuality", "DewTemperature", "DewTemperatureQuality", "RelativeHumidity",
    "RelativeHumidityQuality", "WindSpeed", "WindSpeedQuality", "WindDirection", "WindDirectionQuality", "WindgustSpeed", "WindgustSpeedQuality", "SeaPressure",
    "SeaPressureQuality", "StationPressure", "StationPressureQuality", "AWSFlag", "#"
]

## Remove any easily found unnecessary data
Remove any obvious duplicated data or data which always has the same value (or is just meaningless to analysis).
Briefly taking a look at the data gives an overview of what may easily be removed!

**Start by simply getting an overview of the data, then look at unique values and basic descriptive statistics**.

*Note manual filtering is used for exceptional data*

In [6]:
energy_data
temperature_data
stations_data

,Region,Date,TotalDemand,RRP,PeriodType
0,NSW1,2000/01/01 00:30,6763.57000,15.64,TRADE
1,NSW1,2000/01/01 01:00,6386.10167,14.06,TRADE
2,NSW1,2000/01/01 01:30,5990.79500,14.30,TRADE
3,NSW1,2000/01/01 02:00,5655.97667,14.28,TRADE
4,NSW1,2000/01/01 02:30,5283.83667,14.17,TRADE
...,...,...,...,...,...
1658960,VIC1,2019/12/31 22:00:00,4129.96000,52.91,TRADE
1658961,VIC1,2019/12/31 22:30:00,4083.66000,58.66,TRADE
1658962,VIC1,2019/12/31 23:00:00,4104.95000,54.36,TRADE
1658963,VIC1,2019/12/31 23:30:00,4325.88000,66.87,TRADE


,HM,StationNumber,Year1,Month1,Day1,Hour1,Minute1,Year,Month,Day,Hour,Minute,Precipitation,PrecipitationQuality,AirTemperature,AirTemperatureQuality,WetBulbTemperature,WetBulbTemperatureQuality,DewTemperature,DewTemperatureQuality,RelativeHumidity,RelativeHumidityQuality,WindSpeed,WindSpeedQuality,WindDirection,WindDirectionQuality,WindgustSpeed,WindgustSpeedQuality,SeaPressure,SeaPressureQuality,StationPressure,StationPressureQuality,AWSFlag,#
0,hm,23090,2000,1,1,1,0,2000,1,1,0,0,0,N,13.1,N,10.0,N,6.6,N,65,N,14.8,N,190,N,24.1,N,1018.9,N,1012.8,N,,#
1,hm,23090,2000,1,1,1,30,2000,1,1,0,30,0,N,13.2,N,10.1,N,6.8,N,65,N,9.4,N,200,N,16.6,N,1018.7,N,1012.6,N,,#
2,hm,23090,2000,1,1,2,0,2000,1,1,1,0,0,N,13.4,N,10.1,N,6.5,N,63,N,13,N,180,N,20.5,N,1018.5,N,1012.4,N,,#
3,hm,23090,2000,1,1,2,30,2000,1,1,1,30,0,N,13.2,N,10.0,N,6.5,N,64,N,9.4,N,170,N,18.4,N,1018.3,N,1012.2,N,,#
4,hm,23090,2000,1,1,3,0,2000,1,1,2,0,0,N,13.0,N,9.7,N,6.0,N,62,N,11.2,N,150,N,20.5,N,1018.3,N,1012.2,N,,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826239,hm,94029,2020,1,20,8,0,2020,1,20,7,0,0.0,N,14.9,N,12.5,N,10.3,N,74,N,14.8,N,130,N,20.5,N,1011.4,N,1005.3,N,1,#
1826240,hm,94029,2020,1,20,8,30,2020,1,20,7,30,0.0,N,14.9,N,12.4,N,10.1,N,73,N,13,N,140,N,18.4,N,1011.5,N,1005.4,N,1,#
1826241,hm,94029,2020,1,20,9,0,2020,1,20,8,0,0.0,N,14.9,N,12.4,N,10.1,N,73,N,16.6,N,150,N,24.1,N,1011.7,N,1005.6,N,1,#
1826242,hm,94029,2020,1,20,9,30,2020,1,20,8,30,0.0,N,15.1,N,12.4,N,9.9,N,71,N,16.6,N,150,N,22.3,N,1011.6,N,1005.5,N,1,#


,StationNumber,RainfulDistrict,Station,StationOpenDate,StationCloseDate,Latitude,Longitude,LocationMethod,State,StationHeight,BarometerHeight,WMO,FirstYear,LastYear,CompletionPercent,YQualityPercent,NQualityPercent,WQualityPercent,SQualityPercent,IQualityPercent
0,86338,86,MELBOURNE (OLYMPIC PARK),05/2013,,-37.8255,144.9816,SURVEY,VIC,7.5,7.5,95936,2013,2020,103,0,100,0,0,0
1,86071,86,MELBOURNE REGIONAL OFFICE,01/1908,01/2015,-37.8075,144.9700,GPS,VIC,31.2,32.2,94868,2000,2015,99,0,100,0,0,0
2,23090,23A,ADELAIDE (KENT TOWN),01/1977,,-34.9211,138.6216,GPS,SA,48.0,51.0,94675,2000,2020,101,0,100,0,0,0
3,66062,66,SYDNEY (OBSERVATORY HILL),01/1858,,-33.8607,151.2050,GPS,NSW,39.0,40.2,94768,2000,2020,99,0,100,0,0,0
4,94029,94,HOBART (ELLERSLIE ROAD),01/1882,,-42.8897,147.3278,GPS,TAS,50.5,51.4,94970,2000,2020,110,0,100,0,0,0
5,40913,40,BRISBANE,12/1999,,-27.4808,153.0389,GPS,QLD,8.1,8.3,94576,2000,2020,99,0,100,0,0,0


In [7]:
def remove_non_uniques(dataframe: pd.DataFrame, filter = []):
    remove = [name for name, series in dataframe.items() if len(series.unique()) <= 2 and not name in filter]
    dataframe.drop(remove, axis=1, inplace=True)
    return remove

print("Removed:")
remove_non_uniques(energy_data)
remove_non_uniques(temperature_data)
remove_non_uniques(stations_data, "LocationMethod")

# Manually remove extra columns
stations_data.drop("StationOpenDate", axis=1, inplace=True)

Removed:


['PeriodType']

['HM',
 'PrecipitationQuality',
 'AirTemperatureQuality',
 'WetBulbTemperatureQuality',
 'DewTemperatureQuality',
 'RelativeHumidityQuality',
 'WindSpeedQuality',
 'WindDirectionQuality',
 'WindgustSpeedQuality',
 'SeaPressureQuality',
 'StationPressureQuality',
 '#']

['StationCloseDate',
 'FirstYear',
 'LastYear',
 'YQualityPercent',
 'NQualityPercent',
 'WQualityPercent',
 'SQualityPercent',
 'IQualityPercent']

In [8]:
energy_data.columns
temperature_data.columns
stations_data.columns

Index(['Region', 'Date', 'TotalDemand', 'RRP'], dtype='object')

Index(['StationNumber', 'Year1', 'Month1', 'Day1', 'Hour1', 'Minute1', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Precipitation', 'AirTemperature',
       'WetBulbTemperature', 'DewTemperature', 'RelativeHumidity', 'WindSpeed',
       'WindDirection', 'WindgustSpeed', 'SeaPressure', 'StationPressure',
       'AWSFlag'],
      dtype='object')

Index(['StationNumber', 'RainfulDistrict', 'Station', 'Latitude', 'Longitude',
       'LocationMethod', 'State', 'StationHeight', 'BarometerHeight', 'WMO',
       'CompletionPercent'],
      dtype='object')

## Deal with multiple dates
Find out what the difference between the dates are.
Then ensure all dates are combined into one column.

Provided notes dictate that first set of dates are local and second local standard.
Whilst generally identical, local time incorporates day light saving (shifting the clock back/foward by an hour).
For simplicity, local standard time will be used.
*Note that previouslly the local standard columns were named without a 1, and the others with a 1 to avoid renaming here*!

In [9]:
# Remove extra dates
temperature_data.drop(["Year1", "Month1", "Day1", "Hour1", "Minute1"], axis=1, inplace=True)

# Reformat dates into Pandas' datatime64 objects
# Replacing old format
temperature_data["Date"] = pd.to_datetime(temperature_data[["Year", "Month", "Day", "Hour", "Minute"]])
energy_data["Date"] = pd.to_datetime(energy_data["Date"])

temperature_data.drop(["Year", "Month", "Day", "Hour", "Minute"], axis=1, inplace=True)

## Check for duplicated rows
Note that this process works on *all columns* (as a specific choice hasn't been provided)

To check which rows are removed Pandas duplicated function can be used beforehand.
This has been excluded from analysis for brevity (especially since there are **no found duplicates**)!

In [10]:
energy_data.drop_duplicates(inplace=True)
temperature_data.drop_duplicates(inplace=True)
stations_data.drop_duplicates(inplace=True)

## Converting datatypes
To make sure we can create visualisations/plots and mathematical/statistical models we need to use numerical datatypes (i.e. not objects)!
Note for AWSFlag, empty values are assumed to be manually recorded as this is the worst outcome.

In [11]:
def to_object_columns(lambda_function):
    string_columns = temperature_data.select_dtypes("object").columns
    temperature_data[string_columns] = temperature_data[string_columns].apply(lambda_function)

In [12]:
to_object_columns(lambda column: column.str.strip())

In [13]:
temperature_data["AWSFlag"] = temperature_data["AWSFlag"].replace("", 0).astype("category")
temperature_data["AWSFlag"].fillna(0, inplace=True)
temperature_data["RelativeHumidity"] = temperature_data["RelativeHumidity"].replace("###", np.NaN)

In [14]:
to_object_columns(lambda column: pd.to_numeric(column))

In [15]:
temperature_data.dtypes

StationNumber                  int64
Precipitation                float64
AirTemperature               float64
WetBulbTemperature           float64
DewTemperature               float64
RelativeHumidity             float64
WindSpeed                    float64
WindDirection                float64
WindgustSpeed                float64
SeaPressure                  float64
StationPressure              float64
AWSFlag                     category
Date                  datetime64[ns]
dtype: object

## Finding missing data
Columns without null values are ignored to simplify the output.

Note the energy data has no null values.
This is shown by the empty series (which for the temperature data shows the counts of empty cells for each column).

In [16]:
def get_null_counts(dataframe: pd.DataFrame):
    return dataframe.isnull().mean()[dataframe.isnull().mean() > 0]

In [17]:
get_null_counts(energy_data)
get_null_counts(temperature_data)

Series([], dtype: float64)

Precipitation         0.229916
AirTemperature        0.444437
WetBulbTemperature    0.011324
DewTemperature        0.375311
RelativeHumidity      0.375312
WindSpeed             0.532966
WindDirection         0.432305
WindgustSpeed         0.403183
SeaPressure           0.137730
StationPressure       0.011135
dtype: float64

The cleaned data must be manually saved now so the missing data visualisations can happen

In [18]:
pd.to_pickle([energy_data, temperature_data, stations_data], "../Data/CleanedData.pickle")

From the graphs (in the graphing notebook) we can tell that the wind data is missing in large chunks and so won't be useful for our analysis.
Hence we remove this data.

The graphs also show that it should be safe to interpolate the missing values.
The linear method is used here, but averages/values above or below may also work.

In [19]:
temperature_data.drop(["WindSpeed", "WindgustSpeed", "WindDirection"], axis=1, inplace=True)

# Note that using inplace currently throws an error
# So interpolated columns must be manually overridden
missing_columns = list(get_null_counts(temperature_data).keys())
temperature_data[missing_columns] = temperature_data[missing_columns].interpolate(method="linear")

## Grouping data
Energy and temperature data are recorded in specific regions.
The naming of the two needs to be standardised so they can easily/smoothly be used as Pandas groups!

*Note more than one weather station existed in Victoria in some timeframe and here both stations are effectively merged into one*.
*Although this may lead to a slightly biased dataset, the effect is assumed for the time being to be minimal*.

In [20]:
energy_data["Region"].unique()
temperature_data["StationNumber"].unique()

array(['NSW1', 'QLD1', 'SA1', 'VIC1', 'TAS1'], dtype=object)

array([23090, 40913, 66062, 86071, 86338, 94029], dtype=int64)

In [21]:
region_remove_number_map = {"SA1": "SA", "QLD1": "QLD", "NSW1": "NSW", "VIC1": "VIC", "TAS1": "TAS"}
station_to_region_map = {23090: "SA", 40913: "QLD", 66062: "NSW", 86071: "VIC", 94029: "TAS", 86338: "VIC"}

temperature_data["Region"] = temperature_data["StationNumber"].map(station_to_region_map)
energy_data["Region"] = energy_data["Region"].map(region_remove_number_map)

temperature_data.drop("StationNumber", axis=1, inplace=True)

## Combining energy and temperature data
To merge the two dataframes together, we first ensure that both datasets are sorted according to date, we then merge them according to closest values.
Here the column values to group by must be manually specified (through the *by* parameter) and a reasonable tolerance must be specified (here our data is recorded in 30 minute increments and so this is reasonable)!
Having no or few null values implies that this process has happened relatively successfully.

*Note that we can do this as the frequency of data being recorded (i.e. date) is decently consistant accross both the energy and temperature datasets*!
If this was not the case, down sampling of one dataset would have to occur.

We will remove any left over null values (as there are a relatively small number of them).

In [22]:
energy_data.sort_values(by="Date", inplace=True)
temperature_data.sort_values(by="Date", inplace=True)

data = pd.merge_asof(energy_data, temperature_data, on="Date", by="Region", tolerance=pd.Timedelta("30 min"))
data.set_index("Date", inplace=True)

In [23]:
get_null_counts(data)
data.dropna(inplace=True)

Precipitation         0.011302
AirTemperature        0.001634
WetBulbTemperature    0.001634
DewTemperature        0.001634
RelativeHumidity      0.001634
SeaPressure           0.001634
StationPressure       0.001634
AWSFlag               0.001634
dtype: float64

In [24]:
data

,Region,TotalDemand,RRP,Precipitation,AirTemperature,WetBulbTemperature,DewTemperature,RelativeHumidity,SeaPressure,StationPressure,AWSFlag
Date,,,,,,,,,,,
2000-01-01 00:30:00,NSW,6763.57000,15.64,0.000000,17.841568,14.1,11.821183,68.136687,1011.100000,1006.4,0
2000-01-01 00:30:00,QLD,3905.56833,39.40,0.000000,18.594473,19.2,14.600415,93.998779,1007.200000,1006.2,0
2000-01-01 00:30:00,VIC,4419.03667,9.97,0.000000,13.700000,10.5,7.200000,65.000000,1017.000000,1013.1,0
2000-01-01 01:00:00,VIC,4312.54000,12.58,0.000000,13.600000,10.3,6.700000,63.000000,1016.800000,1012.9,0
2000-01-01 01:00:00,NSW,6386.10167,14.06,0.000000,17.841429,13.7,11.821143,68.137140,1011.000000,1006.3,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 00:00:00,TAS,1006.70000,93.88,0.000000,17.900000,9.5,9.100000,56.000000,1016.300000,1004.6,0
2020-01-01 00:00:00,QLD,6218.39000,52.15,0.000000,17.970884,21.6,11.858802,67.713472,1011.440920,1013.8,0
2020-01-01 00:00:00,SA,1474.11000,85.95,0.000000,18.469005,12.3,8.700000,61.000000,1008.290509,1010.6,0


## Saving final model

In [25]:
pd.to_pickle(data, "../Data/Data.pickle")